- 源码面前，了无秘密
    - 学习成熟的框架，提升代码水平
    - 后续直接改源码

## type


- **task_type/model_type**:

    
    - `LoraConfig(task_type='CAUSAL_LM')`
    
    ```
    MODEL_TYPE_TO_PEFT_MODEL_MAPPING = {
        "SEQ_CLS": PeftModelForSequenceClassification,
        "SEQ_2_SEQ_LM": PeftModelForSeq2SeqLM,
        "CAUSAL_LM": PeftModelForCausalLM,
        "TOKEN_CLS": PeftModelForTokenClassification,
    }
    ```

- **peft_type**

    ```
    PEFT_TYPE_TO_MODEL_MAPPING = {
        PeftType.LORA: LoraModel,
        PeftType.PROMPT_TUNING: PromptEmbedding,
        PeftType.P_TUNING: PromptEncoder,
        PeftType.PREFIX_TUNING: PrefixEncoder,
        PeftType.ADALORA: AdaLoraModel,
        PeftType.ADAPTION_PROMPT: AdaptionPromptModel,
    }
    ```

## `PeftModel`

```
self.base_model = model
self.config = self.base_model.config
# adapter_name="default"
self.active_adapter = adapter_name
```

## LoraConfig 与 LoraModel

### LoraConfig

```
@dataclass
class LoraConfig(PeftConfig):
    r: int = field(default=8, metadata={"help": "Lora attention dimension"})
    target_modules: Optional[Union[List[str], str]]
    lora_alpha: int = field(default=None, metadata={"help": "Lora alpha"})
    lora_dropout: float = field(default=None, metadata={"help": "Lora dropout"})
    
```

- `target_modules`
    - The names of the modules to apply Lora to.
    - default: `query_key_value`
        - `'transformer.h.0.self_attention.query_key_value'`
    - 都意味着一个大矩阵 $W\in \mathbb R^{A\times B}$；
    


```
TRANSFORMERS_MODELS_TO_LORA_TARGET_MODULES_MAPPING = {
    "t5": ["q", "v"],
    "mt5": ["q", "v"],
    "bart": ["q_proj", "v_proj"],
    "gpt2": ["c_attn"],
    "bloom": ["query_key_value"],
    "blip-2": ["q", "v", "q_proj", "v_proj"],
    "opt": ["q_proj", "v_proj"],
    "gptj": ["q_proj", "v_proj"],
    "gpt_neox": ["query_key_value"],
    "gpt_neo": ["q_proj", "v_proj"],
    "bert": ["query", "value"],
    "roberta": ["query", "value"],
    "xlm-roberta": ["query", "value"],
    "electra": ["query", "value"],
    "deberta-v2": ["query_proj", "value_proj"],
    "deberta": ["in_proj"],
    "layoutlm": ["query", "value"],
    "llama": ["q_proj", "v_proj"],
    "chatglm": ["query_key_value"],
}
```

```
key_list = [key for key, _ in self.model.named_modules()]
for key in key_list:
    if isinstance(lora_config.target_modules, str):
        target_module_found = re.fullmatch(lora_config.target_modules, key)
    else:
        target_module_found = any(key.endswith(target_key) for target_key in lora_config.target_modules)
```



In [5]:
from peft import LoraConfig 
config = LoraConfig(
    r=16, #low rank
    lora_alpha=32, #alpha scaling， scale lora weights/outputs
    # target_modules=["q_proj", "v_proj"], #if you know the 
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM" # set this for CLM or Seq2Seq
)

### model => loramodel

In [2]:
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

In [3]:
model = AutoModelForCausalLM.from_pretrained(
    "bigscience/bloom-7b1", 
    load_in_8bit=True, 
    device_map='auto',
)

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
model

BloomForCausalLM(
  (transformer): BloomModel(
    (word_embeddings): Embedding(250880, 4096)
    (word_embeddings_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
    (h): ModuleList(
      (0-29): 30 x BloomBlock(
        (input_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (self_attention): BloomAttention(
          (query_key_value): Linear8bitLt(in_features=4096, out_features=12288, bias=True)
          (dense): Linear8bitLt(in_features=4096, out_features=4096, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (post_attention_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (mlp): BloomMLP(
          (dense_h_to_4h): Linear8bitLt(in_features=4096, out_features=16384, bias=True)
          (gelu_impl): BloomGelu()
          (dense_4h_to_h): Linear8bitLt(in_features=16384, out_features=4096, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((4096,), eps=1e-05, eleme

In [4]:
from peft import get_peft_model

In [7]:
model = get_peft_model(model, config)

In [8]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): BloomForCausalLM(
      (transformer): BloomModel(
        (word_embeddings): Embedding(250880, 4096)
        (word_embeddings_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (h): ModuleList(
          (0-29): 30 x BloomBlock(
            (input_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
            (self_attention): BloomAttention(
              (query_key_value): Linear8bitLt(
                in_features=4096, out_features=12288, bias=True
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=12288, bias=False)
                )
                (lora_embedding_A): Para